In [ ]:
import ijson
import base64
from minio import Minio
from pymongo import MongoClient
import io
from decimal import Decimal

# --- CONFIGURACIÓN ---
input_json = r"C:\Users\Usuario\Documents\GitHub\PFG\AQ_database_filtered_june_2087.json"

# MinIO
minio_client = Minio(
    "localhost:9000",
    access_key="admin",
    secret_key="password123",
    secure=False
)
bucket_name = "pfg"  # <-- aquí cambiamos el bucket

# MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
db = mongo_client["aire"]
collection = db["registros"]

# Crear bucket si no existe
if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)

# --- FUNCIONES DE AYUDA ---
def sanitize_dict(d):
    """Elimina claves que empiezan con $ y convierte Decimals a float recursivamente."""
    if isinstance(d, dict):
        new_d = {}
        for k, v in d.items():
            key = k[1:] if k.startswith("$") else k
            new_d[key] = sanitize_dict(v)
        return new_d
    elif isinstance(d, list):
        return [sanitize_dict(v) for v in d]
    elif isinstance(d, Decimal):
        return float(d)
    else:
        return d

# --- PROCESO ---
with open(input_json, "rb") as f:
    registros = ijson.items(f, "item")  # 'item' si JSON es [ {...}, {...} ]

    for i, registro in enumerate(registros):
        # --- ID para Mongo ---
        id_raw = registro.get("_id", None)
        if isinstance(id_raw, dict) and "$oid" in id_raw:
            registro_db_id = id_raw["$oid"]
        else:
            registro_db_id = str(id_raw or f"img_{i}")

        # --- Imagen ---
        img_b64 = registro.get("Imagen de entrada", "")
        if img_b64:
            if "," in img_b64:
                img_b64 = img_b64.split(",")[1]
            img_bytes = base64.b64decode(img_b64)
            object_name = f"{registro_db_id}.jpg"
            img_stream = io.BytesIO(img_bytes)
            minio_client.put_object(bucket_name, object_name,
                                    data=img_stream, length=len(img_bytes))
            img_url = f"http://localhost:9000/{bucket_name}/{object_name}"
        else:
            img_url = None

        # --- Preparar registro para Mongo ---
        registro_db = {k: registro.get(k, "") for k in registro.keys()}
        registro_db["_id"] = registro_db_id
        registro_db["Imagen_URI"] = img_url

        # Limpiar claves $ y convertir Decimals
        registro_db = sanitize_dict(registro_db)

        # Insertar
        try:
            collection.insert_one(registro_db)
            if (i + 1) % 50 == 0:
                print(f" Procesados {i+1} registros...")
        except Exception as e:
            print(f" Error insertando registro {registro_db_id}: {e}")

print("\n ¡Proceso completado!")


✅ Procesados 50 registros...
✅ Procesados 100 registros...
✅ Procesados 150 registros...
✅ Procesados 200 registros...
✅ Procesados 250 registros...
✅ Procesados 300 registros...
✅ Procesados 350 registros...
✅ Procesados 400 registros...
✅ Procesados 450 registros...
✅ Procesados 500 registros...
✅ Procesados 550 registros...
✅ Procesados 600 registros...
✅ Procesados 650 registros...
✅ Procesados 700 registros...
✅ Procesados 750 registros...
✅ Procesados 800 registros...
✅ Procesados 850 registros...
✅ Procesados 900 registros...
✅ Procesados 950 registros...
✅ Procesados 1000 registros...
✅ Procesados 1050 registros...
✅ Procesados 1100 registros...
✅ Procesados 1150 registros...
✅ Procesados 1200 registros...
✅ Procesados 1250 registros...
✅ Procesados 1300 registros...
✅ Procesados 1350 registros...
✅ Procesados 1400 registros...
✅ Procesados 1450 registros...
✅ Procesados 1500 registros...
✅ Procesados 1550 registros...
✅ Procesados 1600 registros...
✅ Procesados 1650 registros.